In [1]:
import pandas as pd
import numpy as np
print("the code is running")
# from Package.Models import *
import torch
import pickle
from torch import nn
from torch.nn.modules.padding import ConstantPad1d,ReflectionPad1d
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.model_selection import train_test_split
from numba import jit
from matplotlib import pyplot as plt
from torch.autograd import Variable
from sklearn.utils import shuffle
from time import time
from textblob import TextBlob as tb
print("import statements finished")

def load_glove_model(gloveFile):
    import numpy as np
    print ("Loading Glove Model")
    f = open(gloveFile,'r',encoding="utf8")
    print("file read")
    model = {}
    count = 0
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        try:
            embedding = np.array([float(val) for val in splitLine[1:]])
            model[word] = embedding
        except:
            count = count+1
            pass
    print(count)
    print ("Done.",len(model)," words loaded!")
    
    return model
    
    



# from Package.Features import *
# from Package import Newfile as nf
# global X,Y

## loading the glove vectors
embedding_loc = "../input/embeddings/glove.840B.300d/glove.840B.300d.txt"
# embedding_loc = "../input/quora-insincere-questions-classification/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec"
try:
    glv
except:
    glv = load_glove_model(embedding_loc)
print("trying to load vectors")    
print(len(glv))


the code is running
import statements finished
Loading Glove Model
file read
20
Done. 2195884  words loaded!
trying to load vectors
2195884


In [9]:
DEBUG = True
from nltk.tokenize import TweetTokenizer
import re
tknzr = TweetTokenizer()

torch.manual_seed(2)
def preprocess(t):
    
    t = t.encode("ASCII","ignore").decode("utf-8")
    t = " ".join(tknzr.tokenize(t))
    t = str(t).lower()
    t = re.sub(r'["\'|?"]','',t)
    t = re.sub(r'[\-]',' ',t)
#     print(t)
#     print("^^^^^^^^^^^^^^^^^^")
    return t

def log(*argv):
    if DEBUG == True:
        try:
            print(argv)
        except:
            print("Error in printing")

def get_batches(x,y,chunks = 400):
    l = len(x)
    cnt = int(np.round(int(l)/int(chunks)))
    # log(cnt)
    remain = l%chunks
    rmt = remain/cnt
    # log(rmt)
    x_list = [x[each*cnt:(each*cnt)+cnt] for each in range(chunks+int(rmt)+2)]
    y_list = [y[each*cnt:(each*cnt)+cnt] for each in range(chunks+int(rmt)+2)]
    return x_list,y_list


@jit
def numba_mean(loss_list):
    return np.array(loss_list).mean()
    



In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
dtype = torch.FloatTensor
# CUDA_LAUNCH_BLOCKING=1
torch.set_default_tensor_type('torch.FloatTensor')

## Model definition
class encoder_rnn(nn.Module):
    def __init__(self,input_dim, hidden_dim, layer_dim, output_dim):
        super(encoder_rnn,self).__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True,bidirectional = True)
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim
        self.fc = net = nn.Sequential(
                  nn.Linear(hidden_dim*2, hidden_dim*4),
#                   nn.BatchNorm1d(num_features = (hidden_dim*4)),
                  nn.ReLU(),
                  nn.Linear(hidden_dim*4,output_dim),
                  nn.Sigmoid()
                  )
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(num_features = hidden_dim*4)

    def forward(self,x):
        for e in range(len(x)):
            X = (torch.from_numpy(x[e])[None,None,:]).type(dtype)            
            if e == 0:
                h0 = (torch.randn(self.layer_dim*2, 1, self.hidden_dim).type(dtype),torch.randn(self.layer_dim*2, 1, self.hidden_dim).type(dtype))
#                 h0 = torch.randn(self.layer_dim*2, 1, self.hidden_dim).type(torch.cuda.FloatTensor)
                output,hidden = self.rnn(X,h0)
            else:
                output,hidden = self.rnn(X,hidden)
        output = (output).view(-1)
        output = self.relu(output)
        final = self.fc(output)
        return final
def init_weights(m):
    if type(m) == nn.Linear:
        print("Xavier applied")
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)
    
## initiate a model   
model  = encoder_rnn(301,30,4,1)
# torch.nn.init.xavier_uniform(model.weight)
model.apply(init_weights)
# model = model.cuda()

## loading the training data
# q = pd.read_csv("../input/quora-insincere-questions-classification/train.csv",index_col = None,encoding = "iso-8859-1")
q = pd.read_csv("../input/train.csv",index_col = None,encoding = "iso-8859-1")
sin = q[q["target"] == 0]
insin = q[q["target"] == 1]
sin = sin.sample(frac = 0.10,random_state = 42)
q = pd.concat([sin,insin])
q = shuffle(q).reset_index(drop = True)
# q = q[:100000]
log("Total rows ",q.shape)

def get_vectors(text,glv,pad = 250,get_pos = False):
    import nltk
    import numpy as np
    glv_text = " ".join([ each for each in text.split(" ") if each in glv]) 
#     tp = [each for each in glv_text.split(" ")]
#     log(tp)
    all_vectors = [glv[each] for each in glv_text.split(" ")]
    all_vectors = [np.append(all_vectors[i],(tb(each).sentiment[0])) for i,each in enumerate(glv_text.split(" "))]
    temp = np.zeros((301))
    
    if pad != None:
        all_vectors = all_vectors[:pad]
        if len(all_vectors) <pad:
            temp_range = pad-len(all_vectors)
            temp_list = temp.tolist()
            [all_vectors.append(temp_list)for each in range(temp_range)]
    if len(all_vectors)== 0:
        log("The words in this text has no vectors")
        log(text)
        all_vectors.append(temp)
    all_vectors = np.array(all_vectors)
    if get_pos == True:
        pos_vector = get_pos_tag(text,glv)
        if len(all_vectors)!= len(pos_vector):
            print("Pos Error detected")
        all_vectors = [np.concatenate((each,pos_vector[1])) for i,each in enumerate(all_vectors) if len(all_vectors)== len(pos_vector)]
    return all_vectors



log(q.groupby("target")["qid"].count())
log("Considered rows ",q.shape)
q["question_text"] = [ preprocess(each) for each in  q["question_text"]]
# q.preprocess(remove_stopwords = False,lemmatize = False)
log("preprocessing words complete")
q["x"]  = q["question_text"]
q["y"] = q["target"]

## test train split
x,xt,y,yt = train_test_split(q['x'],q['y'],test_size=0.005, random_state=42)
x = x.reset_index(drop = True)
xt = xt.reset_index(drop = True)
y = y.reset_index(drop = True)
yt =yt.reset_index(drop = True)

## readying the vectors for the test data
xt = [get_vectors(each,glv,pad= None) for each in xt]

log("Vectorization complete")

## obtaining batches
batches = 250
x_batches,y_batches = get_batches(x,y,chunks = batches)
x_batches = [each.reset_index(drop = True) for each in x_batches if len(each)!=0]
y_batches = [each.reset_index(drop = True) for each in y_batches if len(each)!=0]

log("splitting the batches is complete")  
log("The shape of each batch is ",x_batches[0].shape)      
log("The shape of test data is ",len(xt))      

def feed(x,y,backprop = False,epoch=1):
    t1 = time()
    loss_list = []
    for i,each in enumerate(x):
        X = each
        Y = (torch.from_numpy(np.array(y[i]))).type(dtype)
        pred = model.forward(X)
        loss =cross(pred,Y).type(dtype)
        if backprop == True:
            loss.backward()
            optimizer.step()
        loss_list.append(loss.item())
    print("T:",str(time()-t1))
    return loss_list

epoch = 1
lr = 0.00001
checkpoint = 3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
optimizer.zero_grad()
cross = nn.BCELoss()

log("Model training starts")

for each in range(epoch):
    for j,e in enumerate(x_batches):
        x, y = x_batches[j],y_batches[j]
        x = [get_vectors(each,glv,pad=None) for each in x]
        train_cost = numba_mean(feed(x,y,backprop = True,epoch = each))  
        
        
        log("Epoch num - ",each," and batch num ",j)
        log(train_cost)
        if j%7 == 0:
            test_cost = numba_mean(feed(xt,yt,backprop = False,epoch = each))
            log(test_cost)        
        log("================================================")
        if j == 40:
            break
    break
print("completed")


Using device: cpu
Xavier applied
Xavier applied


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


('Total rows ', (203341, 3))
(target
0    122531
1     80810
Name: qid, dtype: int64,)
('Considered rows ', (203341, 3))
('preprocessing words complete',)
('Vectorization complete',)
('splitting the batches is complete',)
('The shape of each batch is ', (809,))
('The shape of test data is ', 1017)
('Model training starts',)


/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


T: 35.287957191467285
('Epoch num - ', 0, ' and batch num ', 0)
(0.6720716817343633,)
T: 16.866023063659668
(0.6529606126812941,)
('================================================',)
T: 34.69876289367676
('Epoch num - ', 0, ' and batch num ', 1)
(0.5810174507322653,)
('================================================',)
T: 35.731828689575195
('Epoch num - ', 0, ' and batch num ', 2)
(0.46690129160613963,)
('================================================',)
T: 34.026169300079346
('Epoch num - ', 0, ' and batch num ', 3)
(0.4032530383159705,)
('================================================',)
T: 33.86048936843872
('Epoch num - ', 0, ' and batch num ', 4)
(0.3945135514266853,)
('================================================',)
T: 33.82783842086792
('Epoch num - ', 0, ' and batch num ', 5)
(0.36169750250677946,)
('================================================',)
T: 33.80705189704895
('Epoch num - ', 0, ' and batch num ', 6)
(0.3267751537873243,)
('==============================

KeyError: ''

In [6]:
# test = pd.read_csv("../input/quora-insincere-questions-classification/test.csv",index_col = None)
test = pd.read_csv("../input/test.csv",index_col = None)
test["question_text"] = [preprocess(each) for each in test["question_text"]]
log("preprocessing words complete")

test["x"]  = test["question_text"]


result = []
for i,each in enumerate(test["x"]):
    # print(each)
    xv = get_vectors(each,glv,pad = None)
    pred =  model.forward(xv)
    result.append(pred.item())
    if i%5000 == 0:
        log(i)
        
#%%
    
test["result"] = result
threshold = 0.4
test["prediction"] = [ 1 if each > threshold else 0 for each in test.result]
test[["qid","prediction"]].to_csv("submission.csv",index = None)
print("submitted")

my voice range is a2 - c5 . my chest voice goes up to f4 . included sample in my higher chest range . what is my voice type ?
^^^^^^^^^^^^^^^^^^
how much does a tutor earn in bangalore ?
^^^^^^^^^^^^^^^^^^
what are the best made pocket knives under $ 200-300 ?
^^^^^^^^^^^^^^^^^^
why would they add a hypothetical scenario thats impossible to happen in the link below ? it shows what 800 meters rise in sea level would look like .
^^^^^^^^^^^^^^^^^^
what is the dresscode for techmahindra freshers ?
^^^^^^^^^^^^^^^^^^
how well are you adapting to the trump era ?
^^^^^^^^^^^^^^^^^^
what should be the last thing people do in life ?
^^^^^^^^^^^^^^^^^^
received conditional offer for masters in international business from unsw , monash and macquarie . which one should i choose ?
^^^^^^^^^^^^^^^^^^
what does appareils photo mean in french ?
^^^^^^^^^^^^^^^^^^
is there a system of public interest litigation ( pil ) in canada ?
^^^^^^^^^^^^^^^^^^
what's the best weekend plans in chennai for outside

^^^^^^^^^^^^^^^^^^
why do indian muslims have arabic names such as muhammad , wasim , faiza , etc . , and not indian names such as rohit , rahul , siddharth , neha , etc . ? do indian muslims consider arabic culture as superior to indian culture ?
^^^^^^^^^^^^^^^^^^
what is your story and what have you learned so far ?
^^^^^^^^^^^^^^^^^^
is the assad regime really true ? or is it a democarcy ?
^^^^^^^^^^^^^^^^^^
how can one treat you good ?
^^^^^^^^^^^^^^^^^^
do fc barcelona cruyffistas believe that their philosophy could be applied to other clubs or is it just for fc barcelona ?
^^^^^^^^^^^^^^^^^^
what expenses will be involved to create a simple blog with effective hosting in india ?
^^^^^^^^^^^^^^^^^^
in which period of prehistory , pottery making was started ?
^^^^^^^^^^^^^^^^^^
how do i convince my friends to use quora ?
^^^^^^^^^^^^^^^^^^
what is the relationship between geographic factors and economic activity ?
^^^^^^^^^^^^^^^^^^
what is the simple way to learn all the formula 

^^^^^^^^^^^^^^^^^^
do arab israelis think are being spied by the shin bet ?
^^^^^^^^^^^^^^^^^^
why haven't we google map the whole milky way so we can pretend to navigate around in a rocketship ?
^^^^^^^^^^^^^^^^^^
how do you obtain and maintain a job you're not qualified for ?
^^^^^^^^^^^^^^^^^^
what are some debates on nanotechnology ?
^^^^^^^^^^^^^^^^^^
which of these is the best motorcycle phone mount for 2018 ?
^^^^^^^^^^^^^^^^^^
it's november 7 1941 and you wake up as husband e . kimmel . what would you do ?
^^^^^^^^^^^^^^^^^^
what does it mean when someone is " working on themselves " ?
^^^^^^^^^^^^^^^^^^
does one need to attend jee mains for biotechnology in iit ?
^^^^^^^^^^^^^^^^^^
what is the most propaganda laden hollywood movie you have come across ?
^^^^^^^^^^^^^^^^^^
how would you estimate the probability that project president 0bama was authored by the intelligence community in order to amend the relations with the muslim world ?
^^^^^^^^^^^^^^^^^^
do you ever feel like 

is there a way i can unlock my my-school account without the c2k manager ?
^^^^^^^^^^^^^^^^^^
what is the relevance of national thermal power corporation to indian economy and its future prospects ?
^^^^^^^^^^^^^^^^^^
what is the purpose of developing the sense of smell in humans ?
^^^^^^^^^^^^^^^^^^
can a strong core act as posture support / corset / shapewear ?
^^^^^^^^^^^^^^^^^^
how is it like to witness a civil war in your country ?
^^^^^^^^^^^^^^^^^^
does smoking really have a big affect on lung capacity or is it just that smokers tend to not be very active ?
^^^^^^^^^^^^^^^^^^
if i have a story for a game , what steps should i take to make the story interesting and awesome ?
^^^^^^^^^^^^^^^^^^
how can i open rss file through uc browser ?
^^^^^^^^^^^^^^^^^^
what are the benefits of doing m.tech mathematics from an iit ?
^^^^^^^^^^^^^^^^^^
does the kkk hate britain / british people ?
^^^^^^^^^^^^^^^^^^
are the greeks superior than the turks ? why ?
^^^^^^^^^^^^^^^^^^
what is the di

^^^^^^^^^^^^^^^^^^
which is the best way to study in class 11th ?
^^^^^^^^^^^^^^^^^^
what happens if oil spills into a sewing machine motor ?
^^^^^^^^^^^^^^^^^^
how can i report someone for having a fake academic credential ?
^^^^^^^^^^^^^^^^^^
has anyone ever done a study of who profited from the forced relocation of the niesi in wwii ?
^^^^^^^^^^^^^^^^^^
if you could rename yourself , what would it be ?
^^^^^^^^^^^^^^^^^^
how the seating of scania vs volvo ?
^^^^^^^^^^^^^^^^^^
do some women actually enjoy having an outie belly button during pregnancy ?
^^^^^^^^^^^^^^^^^^
why do i keep feeling like no one is really my friend ?
^^^^^^^^^^^^^^^^^^
my high school classes are too easy for me , and having to waste my days doing nothing is depressing . how can i get more out of my classes ?
^^^^^^^^^^^^^^^^^^
what is vegetative reproduction and what are some examples ?
^^^^^^^^^^^^^^^^^^
are twin flames real and how will they affect the flame that's in bad marriage ?
^^^^^^^^^^^^^^^^^^
whic

...

^^^^^^^^^^^^^^^^^^
what are some thing that happen only in india ?
^^^^^^^^^^^^^^^^^^
what can i do if my air conditioner is running but there is no blowing air ?
^^^^^^^^^^^^^^^^^^
is china very developed ?
^^^^^^^^^^^^^^^^^^
what benefits did india got from modi foreign visits ?
^^^^^^^^^^^^^^^^^^
what should i answer in the interview about the years i have wasted ?
^^^^^^^^^^^^^^^^^^
how can i transfer share certificate if my mother is dead i've four brothers whom i don't want to give anything from those shares . what can i do ?
^^^^^^^^^^^^^^^^^^
what will be upcoming trends of purging compounds market ?
^^^^^^^^^^^^^^^^^^
why does elastic potential energy does not depend on mass ?
^^^^^^^^^^^^^^^^^^
why is everything made in china , except babies which are made in va-gina ?
^^^^^^^^^^^^^^^^^^
what is a math researcher's daily life like during the week ?
^^^^^^^^^^^^^^^^^^
what is the syllabus of cet gadvasu ?
^^^^^^^^^^^^^^^^^^
what are some positive things about pursuing a phd ?


over the course of your career , what lessons did you learn that have served you well in life ?
^^^^^^^^^^^^^^^^^^
what is the physical meaning of contravariant and covariant tensor ?
^^^^^^^^^^^^^^^^^^
do focal points order the cosmos ?
^^^^^^^^^^^^^^^^^^
what's the iq level of einstein ?
^^^^^^^^^^^^^^^^^^
what is the london " cycling chief " and how will he fix the " problem " of " too many " white males people riding bicycles ?
^^^^^^^^^^^^^^^^^^
how do you breed african horses ?
^^^^^^^^^^^^^^^^^^
what are the benefits of eating canned food during pregnancy ?
^^^^^^^^^^^^^^^^^^
is delhi institute of rural development better than cpj for bba llb , how good are the placements in both of these ?
^^^^^^^^^^^^^^^^^^
if a nuclear disaster similar to a chernobyl happened in the u . s . , how would they cope ?
^^^^^^^^^^^^^^^^^^
how different is listening to books on audible ( , blinklist ) compared to reading books ?
^^^^^^^^^^^^^^^^^^
what kind of jobs are available for a us citizen mov

how many cm are there in 1000 km between two places ?
^^^^^^^^^^^^^^^^^^
why dont people understand you already have the right to keep and bear arms ? it is the governments job to protect that right .
^^^^^^^^^^^^^^^^^^
what is medically worse , being overweight or smoking ?
^^^^^^^^^^^^^^^^^^
how do i best learn to save money , manage it , and spend it based on an income ? how do i manage how much i spend given it's impossibility to track every penny you spend ?
^^^^^^^^^^^^^^^^^^
why dont bullied children wear bomb vests to school ?
^^^^^^^^^^^^^^^^^^
got 135000 in jee mains 2017 , can i get bit patna ( cse , it or ece ) ?
^^^^^^^^^^^^^^^^^^
immunotherapy seems promising as a better way to treat cancer and other diseases , but might it pose a danger if it attacks the patient's body too ?
^^^^^^^^^^^^^^^^^^
how do i say what i want to say without my voice just getting stuck inside my head ?
^^^^^^^^^^^^^^^^^^
how can grocery shopping be more like a fun activity ?
^^^^^^^^^^^^^^^^^^
ho

why are the american people and politics so polarised ?
^^^^^^^^^^^^^^^^^^
how did they fake sandy hook ?
^^^^^^^^^^^^^^^^^^
can loan account be transferred from one branch to the other in state bank of india ?
^^^^^^^^^^^^^^^^^^
how do i write dvd without any softwares ?
^^^^^^^^^^^^^^^^^^
what attracts you to a woman ?
^^^^^^^^^^^^^^^^^^
what are the exams can i write after preparing for cat ? can i write pgcet or kmat ?
^^^^^^^^^^^^^^^^^^
is it legal in the us to make you purchase an additional product or service in order to purchase a service you want ?
^^^^^^^^^^^^^^^^^^
what is da in salary ?
^^^^^^^^^^^^^^^^^^
is the moderation on stack exchange more transparent than that of quora's ?
^^^^^^^^^^^^^^^^^^
how are the canadian system of government similar to great britain ?
^^^^^^^^^^^^^^^^^^
how should one tackle with the so called anti romeo squads that harass the couples in public places in uttar pradesh ?
^^^^^^^^^^^^^^^^^^
what is the information that i should provide regardin

KeyboardInterrupt: 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
